# [IAPR 2020:][iapr2020] Project ‒ Special Project¶
**Author:** Quentin Talon & Albéric de Lajarte  
**Due date:** 28.05.2020  
[iapr2018]: https://github.com/LTS5/iapr-2020

## Extract datas
We used pims : `pip install git+https://github.com/soft-matter/pims.git`


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pims

data_base_path = os.path.join(os.pardir, 'data')
vid = pims.open(os.path.join(data_base_path, 'robot_parcours_1.avi'))
plt.figure(figsize = (10,10))
plt.imshow(vid[0])
print(vid)

## Part 1: Still and moving image segmentation

### Background
We simply calculate the median along all the images. This reduces the noise and removes the moving object.

In [ ]:
vid_stack = np.stack(vid, axis=0)
print("Shape of vid_stack:{}".format(vid_stack.shape))
background = np.median(vid_stack, axis=0).astype(int)
print("Shape of background:{}".format(background.shape))
plt.figure(figsize = (20,20))
plt.imshow(background)
plt.show()

### Moving part
We look at the red color, clean it a bit and look at his position, size and direction.  


In [ ]:
#Few tests.
from skimage.morphology import binary_opening, binary_closing, disk, label
from skimage.measure import regionprops

col_threshold = 100#How to choose it ???
fig, ax = plt.subplots(1,2, figsize=(20,10))
def red(v, th):
    r = v[:,:,0] > th
    g = v[:,:,1] < th
    b = v[:,:,2] < th
    return np.logical_and(r, g, b)
red_arrow = red(vid[19], col_threshold)
red_arrow_opened = binary_opening(red_arrow, selem=disk(2))

ax[0].imshow(red_arrow)
ax[0].set_title("Before opening dif")
ax[1].imshow(red_arrow_opened)
ax[1].set_title("After opening")
plt.show()


We had to create a home made rectangle mask

In [ ]:
from skimage.draw import polygon
def rect2mask(x0, y0, l_x, l_y, ang, shape):#ang in rad
    hyp = 0.5*np.sqrt(l_x**2+l_y**2)
    rect_ang = np.arctan(l_x/l_y)
    top_right = (x0+np.cos(ang+rect_ang)*hyp, y0+np.sin(ang+rect_ang)*hyp)
    top_left = (x0+np.cos(ang+np.pi-rect_ang)*hyp, y0+np.sin(ang+np.pi-rect_ang)*hyp)
    bottom_right = (x0+np.cos(ang-rect_ang)*hyp, y0+np.sin(ang-rect_ang)*hyp)
    bottom_left = (x0+np.cos(ang+np.pi+rect_ang)*hyp, y0+np.sin(ang+np.pi+rect_ang)*hyp)
    poly_coordinates = np.asarray([top_right, top_left, bottom_left, bottom_right])
    rr, cc = polygon(poly_coordinates[:,0], poly_coordinates[:,1], shape=shape)
    mask = np.zeros(shape, dtype=bool)
    mask[rr, cc, :] = [True, True, True]
    return mask

We create a mask for each frame and apply it on the background for each frame.  
So we have an image of the background were all what is out of the robot is set to 0.

In [ ]:
trajectory = [regionprops(label(binary_opening(red(v, col_threshold), disk(2))), coordinates='xy')[0] for v in vid]
print(vid.frame_shape)
trajectory_mask = [rect2mask(*point.centroid, point.major_axis_length, point.minor_axis_length, point.orientation, vid.frame_shape) for point in trajectory]
print(trajectory_mask[0].shape)
hidden_background = [np.multiply(background,t) for t in trajectory_mask]
plt.imshow(hidden_background[12])
plt.show()

## Albéric's part

In [ ]:
import matplotlib.animation as animation
import matplotlib as mpl
def ani_frame(my_list):
    mpl.rcParams['savefig.pad_inches'] = 0
    fig = plt.figure( frameon=False)
    ax = plt.axes([0,0,1,1], frameon=False)
    ax.set_aspect('auto')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.autoscale(tight=True)
    im = ax.imshow(my_list[0])
    fig.set_size_inches((vid.frame_shape[1]/100, vid.frame_shape[0]/100))
    def update_img(n):
        im.set_data(my_list[n])
        return im
    #legend(loc=0)
    ani = animation.FuncAnimation(fig,update_img,range(len(my_list)))
    writer = animation.writers['ffmpeg'](fps=vid.frame_rate)
    ani.save('demo.mp4',writer=writer,dpi=100)
    return ani

In [ ]:
ani_frame(hidden_background)
print(vid)
vidi = pims.open("demo.mp4")
print(vidi)